In [1]:
import requests
from urllib.parse import quote
from alive_progress import alive_it

## List all pages in category

In [2]:
API_NAME = "vehicles"
FORMAT = "vehicle"

In [3]:
r = requests.get(f"https://api.star-citizen.wiki/api/v2/{API_NAME}/?limit=10000&locale=en_EN").json()["data"]

In [4]:
r = [(component["uuid"], component["name"]) for component in r if component["name"] != "<= PLACEHOLDER =>" and (component["uuid"] is not None)]

In [5]:
r

[('6135a874-4cb1-4f49-9f29-5781e5991f2b', '100i'),
 ('e7eeba8d-b108-48a1-a332-d9edbab2b9c1', '125a'),
 ('2f231daa-95e5-4cb2-a32e-4db8f29a30d6', '135c'),
 ('ce937681-caa2-4cfa-ab80-f5bf2a5b9a6c', '300i'),
 ('dc537d66-2abb-43aa-9d37-4e963fc6e9e2', '315p'),
 ('039a2768-1aff-4969-b2a4-ed8aeb21dae1', '325a'),
 ('316c6601-b405-4a3a-abac-394c655718f8', '350r'),
 ('718f4435-628b-4436-bb80-bf9cfef9a945', '400i'),
 ('f8ac4c26-8f4d-4cb3-9be2-7d867a95a877', '600i Explorer'),
 ('11b660a2-c77c-490b-9e85-ee70400c5cf8', '600i Touring'),
 ('9dc9c257-c413-416a-ab9b-06fbadf18d1e', '85X'),
 ('50ddaf5e-a8e3-42fe-a1d3-7267f39b39b0', '890 Jump'),
 ('c0b86c69-55ac-4af3-a722-d46282813e91', 'A1 Spirit'),
 ('3bee9f2d-4494-4560-b9f8-463bc90695cb', 'A2 Hercules'),
 ('50f551e3-3435-4cbe-9ecf-8712855a8e27', 'Anvil Ballista Dunestalker'),
 ('aaeb976b-7a79-4a9c-91ee-07ed05d4a399', 'Anvil Ballista Snowblind'),
 ('04571d49-0680-4b84-bdb1-dbebeb22c898', 'Ares Inferno'),
 ('30aab266-9131-4a07-808b-61c868cb404f', 'Ares Ion

## Connect to DB

In [6]:
import redis

db = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)

## Add each one to DB

In [7]:
from parse import parse
for (uuid, name) in alive_it(r, force_tty=True):
    data = requests.get(f"https://api.star-citizen.wiki/api/v2/{API_NAME}/" + uuid, params={"locale": "en_EN", "include": "hardpoints,shops,shops.items"}).json()["data"]
    try:
        plaintext = parse(data, FORMAT)
        #print(plaintext)
    except Exception as e:
        print(f"Error parsing: {name}: {e}")
        continue
    db.json().set(f"docs:{name}", redis.commands.json.path.Path.root_path(), {
        "title": name,
        "content": plaintext,
        "url": "example.com"
    })

on 13: Error parsing: A2 Hercules: 'children'                                   
on 43: Error parsing: Constellation Aquila: 'children'                          
on 46: Error parsing: Constellation Taurus: 'item'                              
on 63: Error parsing: F7A Hornet: 'zero_to_scm'                                 
on 82: Error parsing: Glaive: 'children'                                        
on 88: Error parsing: Hull C: 'item'                                            
on 91: Error parsing: Javelin: 'children'                                       
on 106: Error parsing: Mustang Gamma: 'children'                                
on 107: Error parsing: Mustang Omega: 'children'                                
on 123: Error parsing: Reliant Kore: 'children'                                 
on 124: Error parsing: Reliant Mako: 'children'                                 
on 125: Error parsing: Reliant Sen: 'children'                                  
on 135: Error parsing: Scyth

## Do a test query

In [8]:
from redis.commands.search.query import Query
from json import loads
index = db.ft("idx:docs")

In [11]:
res = index.search(Query('hercules'))
print(loads(res.docs[0].json)["content"])

# Ship Stats: C2 Hercules
Dimensions: 70m length, 94m width, 23m height
Emissions: IR: 12849, EM Idle: 21080, EM Max: 60326
Mass: 3139240 kg
Cargo capacity: 696 SCU
Vehicle Inventory: 672
Personal Inventory: 18
Crew: Skeleton: 2, Combat: 3
Health: 81400
Shield HP: 200000 (From shield generator)
Speed: SCM: 135 m/s, Max: 963 m/s, 0 to SCM: 2.536 s, 0 to Max: 18.093 s, SCM to 0: 23.963 s, Max to 0: 170.942 s
Fuel: Capacity: 13155000L, Intake Rate: 0L/s, Usage: Main: 1526L/s, Maneuvering: 886L/s, Retro: 93L/s, VTOL: 0L/s
Quantum: Speed: 59588720km/s, Spool Time: 19.375 s, Fuel Capacity: 88000L, Range: 2277432712215.321 m
Agility: Pitch: 28°/s, Yaw: 24°/s, Roll: 80°/s, Acceleration: Main: 53.223 m/s², Retro: 5.633 m/s², VTOL: 18.455 m/s², Maneuvering: 90.323 m/s²
Armor: IR: 1, EM: 1, Cross Section: 1
Focus: Transport
Role: transport
Description: Utilizing the patented Hercules military-grade spaceframe and expanding cargo capacity, while sacrificing barely any firepower, the C2 has taken t